In [2]:
import os
import random
import shutil
import zipfile
import yaml
import torch
from pathlib import Path
from PIL import Image

def count_images(dataset_path, classes):
    """Counts images for each class to check dataset balance."""
    counts = {class_name: 0 for class_name in classes}
    for class_name in classes:
        class_path = os.path.join(dataset_path, class_name)
        if os.path.exists(class_path):
            counts[class_name] = len([img for img in os.listdir(class_path) if img.lower().endswith(('jpg', 'jpeg', 'png'))])
    return counts

def create_labels(img_path, class_id, subset, base_dir):
    """Create improved labels, ensuring fire engines have better detection coverage."""
    img = Image.open(img_path)
    img_w, img_h = img.size

    if class_id == 1:
        box_w = img_w * 0.7
        box_h = img_h * 0.6
    else:
        box_w = img_w * 0.5
        box_h = img_h * 0.5

    x_center, y_center = 0.5, 0.5
    w, h = box_w / img_w, box_h / img_h

    label_path = os.path.join(base_dir, 'labels', subset, os.path.splitext(os.path.basename(img_path))[0] + '.txt')
    with open(label_path, 'w') as f:
        f.write(f"{class_id} {x_center} {y_center} {w} {h}\n")

def setup_yolo_directory():
    base_dir = '/content/emergency_vehicles'
    for subdir in ['images/train', 'images/val', 'labels/train', 'labels/val']:
        os.makedirs(os.path.join(base_dir, subdir), exist_ok=True)
    return base_dir

def organize_dataset(dataset_path):
    classes = {'Police': 0, 'Fire_Engine': 1, 'Ambulance': 2}
    base_dir = setup_yolo_directory()
    dataset_path = os.path.join(dataset_path, 'Dataset')

    counts = count_images(dataset_path, classes)
    print("Dataset class distribution:", counts)

    dataset_config = {
        'path': base_dir,
        'train': 'images/train',
        'val': 'images/val',
        'names': list(classes.keys()),
        'nc': len(classes)
    }
    with open(f'{base_dir}/dataset.yaml', 'w') as f:
        yaml.dump(dataset_config, f)

    for class_name in classes:
        source_dir = os.path.join(dataset_path, class_name)
        if os.path.exists(source_dir):
            for img in os.listdir(source_dir):
                if img.lower().endswith(('.jpg', '.jpeg', '.png')):
                    is_train = random.random() < 0.8
                    subset = 'train' if is_train else 'val'
                    img_src_path = os.path.join(source_dir, img)
                    dest_path = os.path.join(base_dir, 'images', subset, img)
                    shutil.copy2(img_src_path, dest_path)
                    create_labels(img_src_path, classes[class_name], subset, base_dir)

def setup_yolo():
    from ultralytics import YOLO
    model = YOLO('yolov8n.pt')  # Using YOLOv8 Nano for efficiency
    return model

def train_model(model, base_dir):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    print("Phase 1: Initial Training...")
    model.train(
        data=f'{base_dir}/dataset.yaml',
        epochs=25,
        batch=32,
        imgsz=640,
        save=True,
        project='/content/drive/MyDrive/yolo_training',
        name='phase1_initial',
        save_period=5,
        device=device,
        optimizer='AdamW',  # Optimized for GPU
        lr0=0.01,
        workers=8,
        amp=True,  # Mixed Precision Training
        cache=True,  # Speed up training
        mosaic=0.5
    )

    print("\nPhase 2: Reward Phase - Fire Engine Emphasis...")
    model.train(
        data=f'{base_dir}/dataset.yaml',
        epochs=30,
        batch=32,
        imgsz=640,
        save=True,
        project='/content/drive/MyDrive/yolo_training',
        name='phase2_reward',
        save_period=5,
        device=device,
        optimizer='AdamW',
        lr0=0.005,
        workers=8,
        amp=True,
        cache=True,
        mosaic=0.7,
        box=10.0,
        cls=3.0
    )

    print("\nPhase 3: Fine-tuning Fire Engine Detection...")
    model.train(
        data=f'{base_dir}/dataset.yaml',
        epochs=20,
        batch=32,
        imgsz=640,
        save=True,
        project='/content/drive/MyDrive/yolo_training',
        name='phase3_finetune',
        save_period=5,
        device=device,
        optimizer='AdamW',
        lr0=0.001,
        workers=8,
        amp=True,
        cache=True,
        mosaic=0.3,
        box=5.0,
        cls=3.5
    )

def main():
    from google.colab import drive
    drive.mount('/content/drive')

    print("Installing dependencies...")
    !pip install -q ultralytics

    zip_path = '/content/drive/MyDrive/Dataset.zip'
    extract_path = '/content/Dataset'

    print("Extracting dataset...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    base_dir = setup_yolo_directory()
    organize_dataset(extract_path)
    model = setup_yolo()
    train_model(model, base_dir)
    print("\nTraining completed! Models saved in /content/drive/MyDrive/yolo_training/")

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Installing dependencies...
Extracting dataset...
Dataset class distribution: {'Police': 575, 'Fire_Engine': 800, 'Ambulance': 279}
Using device: cuda
Phase 1: Initial Training...
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/emergency_vehicles/dataset.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=5, cache=True, device=cuda, workers=4, project=/content/drive/MyDrive/yolo_training, name=phase1_initial4, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plo

train: Scanning /content/emergency_vehicles/labels/train... 1377 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1378/1378 [00:00<00:00, 1572.72it/s]

train: WARNING ⚠️ /content/emergency_vehicles/images/train/Screenshot 2024-09-23 110632.png: ignoring corrupt image/label: image size (1, 1) <10 pixels


train: New cache created: /content/emergency_vehicles/labels/train.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (1.1GB RAM): 100%|██████████| 1377/1377 [00:04<00:00, 287.68it/s]


...

      15/15      2.24G     0.3929      2.469      1.183          1        640: 100%|██████████| 87/87 [00:23<00:00,  3.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:03<00:00,  5.66it/s]

                   all        629        629      0.992      0.981      0.993      0.963



15 epochs completed in 0.131 hours.
Optimizer stripped from /content/drive/MyDrive/yolo_training/phase3_finetune/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/yolo_training/phase3_finetune/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/yolo_training/phase3_finetune/weights/best.pt...
Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:07<00:00,  2.78it/s]


                   all        629        629      0.992      0.981      0.993      0.963
                Police        311        311      0.984       0.97      0.991      0.953
           Fire_Engine        149        149      0.998          1      0.995      0.985
             Ambulance        169        169      0.994      0.974      0.994      0.952
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to /content/drive/MyDrive/yolo_training/phase3_finetune

Training completed! Models saved in /content/drive/MyDrive/yolo_training/


In [11]:
#!/usr/bin/env python3
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from google.colab import drive
import os

def mount_drive():
    """Mount Google Drive"""
    drive.mount('/content/drive')

def process_video(video_path, model, output_path):
    """Process video file for vehicle detection and save key frames"""
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Create output paths in Drive
    output_video = str(output_path / f'{video_path.stem}_detected.mp4')
    out = cv2.VideoWriter(output_video,
                         cv2.VideoWriter_fourcc(*'mp4v'),
                         fps,
                         (frame_width, frame_height))

    frames_dir = output_path / f'{video_path.stem}_frames'
    frames_dir.mkdir(exist_ok=True, parents=True)

    frame_count = 0
    save_interval = 30

    class_detections = {
        'Police': 0,
        'Fire_Engine': 0,
        'Ambulance': 0
    }

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            print(f"\rProcessing frame {frame_count}", end="")

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = model(frame_rgb, conf=0.3, iou=0.45)

            for result in results:
                for box in result.boxes:
                    cls = int(box.cls[0])
                    class_names = ['Police', 'Fire_Engine', 'Ambulance']
                    class_name = class_names[cls]
                    class_detections[class_name] += 1

            detected_frame = plot_detection(frame_rgb, results)
            detected_frame = cv2.cvtColor(detected_frame, cv2.COLOR_RGB2BGR)
            out.write(detected_frame)

            if frame_count % save_interval == 0:
                frame_path = frames_dir / f'frame_{frame_count:04d}.jpg'
                cv2.imwrite(str(frame_path), detected_frame)

                print(f"\nDetections in frame {frame_count}:")
                for result in results:
                    for box in result.boxes:
                        conf = float(box.conf[0])
                        cls = int(box.cls[0])
                        class_names = ['Police', 'Fire_Engine', 'Ambulance']
                        class_name = class_names[cls]
                        print(f"- Detected {class_name} with confidence: {conf:.2f}")

    except Exception as e:
        print(f"\nError processing video: {e}")

    finally:
        print(f"\nProcessed {frame_count} frames")
        print(f"Saved detected video to {output_video}")
        print(f"Saved key frames to {frames_dir}")

        print("\nDetection Statistics:")
        for class_name, count in class_detections.items():
            print(f"{class_name}: {count} detections")

        cap.release()
        out.release()

def plot_detection(image, results):
    """Plot detection results"""
    img = image.copy()
    class_names = ['Police', 'Fire_Engine', 'Ambulance']
    colors = {
        'Police': (255, 0, 0),      # Blue
        'Fire_Engine': (0, 165, 255), # Orange
        'Ambulance': (0, 255, 0)    # Green
    }

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            class_name = class_names[cls]
            color = colors[class_name]

            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            label = f'{class_name} {conf:.2f}'
            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
            cv2.rectangle(img, (x1, y1-text_height-8), (x1+text_width, y1), color, -1)
            cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    return img

def process_image(img_path, model, output_path):
    """Process single image"""
    try:
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"Error: Could not read image {img_path}")
            return None

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = model(img, conf=0.3)
        detected_img = plot_detection(img, results)

        output_file = output_path / f'{img_path.stem}_detected.jpg'
        plt.imsave(str(output_file), detected_img)

        print(f"\nResults for {img_path.name}:")
        for result in results:
            for box in result.boxes:
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                class_names = ['Police', 'Fire_Engine', 'Ambulance']
                class_name = class_names[cls]
                print(f"- Detected {class_name} with confidence: {conf:.2f}")

        return output_file

    except Exception as e:
        print(f"Error processing image {img_path}: {e}")
        return None

def main():
    # Mount Google Drive
    mount_drive()

    # Set paths for Google Colab
    model_path = Path('/content/drive/MyDrive/yolo_training/phase3_finetune/weights/best.pt')
    output_path = Path('/content/drive/MyDrive/yolo_results')

    # Create output directory
    output_path.mkdir(parents=True, exist_ok=True)

    if not model_path.exists():
        print(f"Error: Model not found at {model_path}")
        return

    print(f"Loading model from: {model_path}")
    model = YOLO(str(model_path))

    # Process specific WhatsApp videos
    video_files = [
        '/content/Fire engine in Delhi： Traffic on a winter morning in Delhi [Way0mXOFXk0].mp4'
    ]

    for i, video_path in enumerate(video_files, 1):
        video_path = Path(video_path)
        if video_path.exists():
            print(f"\nProcessing video {i}/2: {video_path.name}")
            process_video(video_path, model, output_path)
        else:
            print(f"Video not found: {video_path}")

    print(f"\nAll results saved to {output_path}")

if __name__ == '__main__':
    main()

Streaming output truncated to the last 5000 lines.
0: 384x640 1 Police, 19.8ms
Speed: 3.7ms preprocess, 19.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2483
0: 384x640 1 Police, 24.7ms
Speed: 5.6ms preprocess, 24.7ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2484
0: 384x640 1 Police, 17.4ms
Speed: 5.8ms preprocess, 17.4ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2485
0: 384x640 1 Police, 11.7ms
Speed: 9.9ms preprocess, 11.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2486
0: 384x640 1 Police, 16.0ms
Speed: 5.6ms preprocess, 16.0ms inference, 6.5ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2487
0: 384x640 1 Police, 12.4ms
Speed: 10.5ms preprocess, 12.4ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2488
0: 384x640 1 Police, 16.7ms
Speed: 5.2ms preprocess, 16.7ms infere